In [1]:
import sys
sys.path.append("/opt/drake/lib/python3.10/site-packages")  # The jammy dockerfile needs this.

import numpy as np
from pydrake.all import RobotDiagramBuilder, HybridMultibodyCollocation, Solve, eq
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    DirectCollocation,
    FiniteHorizonLinearQuadraticRegulatorOptions,
    FiniteHorizonLinearQuadraticRegulator,
    LogVectorOutput,
    MakeFiniteHorizonLinearQuadraticRegulator,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePolynomial,
    PlanarSceneGraphVisualizer,
    SceneGraph,
    Simulator,
    Solve,
    TrajectorySource,
)

from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat,
                         MultibodyPlant, InitializeAutoDiffTuple, ExtractGradient, TrajectorySource,
                         FixedOffsetFrame, QuaternionFloatingJoint, UniformGravityFieldElement,
                         ExternallyAppliedSpatialForce, AbstractValue, ContactModel, GeometrySet,
                         CollisionFilterDeclaration, ContactResults, SpatialAcceleration, SpatialForce,
                         SceneGraph, DiagramBuilder_, AutoDiffXd)

from pydrake.common import temp_directory
from pydrake.geometry import (
    MeshcatVisualizerParams,
    Role,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.systems.framework import BasicVector, LeafSystem
import pydrake.symbolic as sym
import pydrake.autodiffutils as autodiff
from pydrake.trajectories import PiecewisePolynomial

np.set_printoptions(suppress=True, precision=3)

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()
meshcat.Set2dRenderMode()

Installing NginX server for MeshCat on Deepnote...
INFO:drake:Meshcat listening for connections at https://067a6c9a-b930-4c28-8084-e86f86716a80.deepnoteproject.com/7000/


## URDFs

In [14]:
climber = """
<?xml version="1.0"?><robot name="Climber">
  <link name="world">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 0" />
      </material>
    </visual>
  </link>

  <link name="hand">
    <inertial>
      <origin xyz="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="1 0.647 0 1" />
      </material>
    </visual>

    <collision>
      <geometry>
        <sphere radius=".05" />
      </geometry>
    </collision>
  </link>

  <link name="arm0">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm1">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm2">
    <inertial>
      <origin xyz="0 0 -0.4" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.8" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.4" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.8"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm3">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm4">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="0.1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <joint name="world-hand" type="planar">
    <parent link="world" />
    <child link="hand" />
    <axis xyz="0 1 0" />
  </joint>
  
  <joint name="theta0" type="continuous">
    <parent link="hand" />
    <child link="arm0" />
    <axis xyz="0 -1 0" />
  </joint>

  <joint name="theta1" type="revolute">
    <origin xyz="0 0 -0.5" rpy="0 0 0"/>
    <parent link="arm0" />
    <child link="arm1" />
    <axis xyz="0 -1 0" />
    <limit effort="10000.0" lower="-2.8" upper="0.0" velocity="10000.0"/>
  </joint>

  <joint name="theta2" type="revolute">
    <origin xyz="0 0 -0.5" rpy="0 0 0"/>
    <parent link="arm1" />
    <child link="arm2" />
    <axis xyz="0 -1 0" />
    <limit effort="10000.0" lower="-0.3" upper="3.4" velocity="10000.0"/>
  </joint>

  <joint name="theta3" type="revolute">
    <origin xyz="0 0 -0.8" rpy="0 0 0"/>
    <parent link="arm2" />
    <child link="arm3" />
    <axis xyz="0 -1 0" />
    <limit effort="10000.0" lower="0.0" upper="2.8" velocity="10000.0"/>
  </joint>

  <joint name="theta4" type="revolute">
    <origin xyz="0 0 -0.5" rpy="0 0 0"/>
    <parent link="arm3" />
    <child link="arm4" />
    <axis xyz="0 -1 0" />
    <limit effort="10000.0" lower="-2.8" upper="0.0" velocity="10000.0"/>
  </joint>
  
  <transmission type="SimpleTransmission" name="theta1_force">
    <actuator name="force1" />
    <joint name="theta1" />
  </transmission>

  <transmission type="SimpleTransmission" name="theta2_force">
    <actuator name="force2" />
    <joint name="theta2" />
  </transmission>

  <transmission type="SimpleTransmission" name="theta3_force">
    <actuator name="force3" />
    <joint name="theta3" />
  </transmission>

  <transmission type="SimpleTransmission" name="theta4_force">
    <actuator name="force4" />
    <joint name="theta4" />
  </transmission>
  
  </robot>
</xml>
"""

In [4]:
obstacle_background = """
<?xml version="1.0"?><robot name="Obstacle_Background">
  <link name="obstacle_base_background">
    <visual>
      <origin xyz="0.125 200 -1" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 1 4"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>
  </link>

  <link name="obstacle_arms_background">
    <visual>
      <origin xyz="0 200 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 -1.0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 -1.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 -2.0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>
  </link>
  
  <joint name="obstacle_base_glue" type="fixed">
    <origin xyz="0 0 0" rpy="0 0 0"/>
    <parent link="obstacle_base_background" />
    <child link="obstacle_arms_background" />
    <axis xyz="0 -1 0" />
  </joint>
  </robot>
</xml>
"""

In [5]:
obstacle = """
<?xml version="1.0"?><robot name="Obstacle">
  <link name="obstacle_base">
    <visual>
      <origin xyz="0.125 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 100 2"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <collision name="obstacle_base_collision">
      <origin xyz="0.125 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 100 2"/>
      </geometry>
    </collision>
  </link>

  <link name="obstacle_arms">
    <visual>
      <origin xyz="0 0 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <collision name="obstacle_top_arm">
      <origin xyz="0 0 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
    </collision>

    <collision name="obstacle_middle_arm">
      <origin xyz="0 0 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
    </collision>
  </link>
  
  <joint name="obstacle_glue" type="fixed">
    <origin xyz="0 0 0" rpy="0 0 0"/>
    <parent link="obstacle_base" />
    <child link="obstacle_arms" />
    <axis xyz="0 -1 0" />
  </joint>

  </robot>
</xml>
"""

## Testing Stuff

In [15]:
# visualizer = ModelVisualizer(meshcat=meshcat)
# visualizer.parser().AddModelsFromString(climber, "urdf")
# visualizer.parser().AddModelsFromString(obstacle_background, "urdf")
# visualizer.parser().AddModelsFromString(obstacle, "urdf")
# visualizer.Run(loop_once=False)
# meshcat.Delete()
# meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


## Solve

In [16]:
def HybridClimber():
    builder = RobotDiagramBuilder()
    
    # Load the climber 
    builder.parser().AddModelsFromString(climber, "urdf")

    # Load the obstacle
    builder.parser().AddModelsFromString(obstacle_background, "urdf")
    builder.parser().AddModelsFromString(obstacle, "urdf")

    plant = builder.plant()
    scene_graph = builder.scene_graph()

    obstacle_frame = plant.GetFrameByName("obstacle_base")
    plant.WeldFrames(plant.world_frame(), obstacle_frame)
    obstacle_background_frame = plant.GetFrameByName("obstacle_base_background")
    plant.WeldFrames(plant.world_frame(), obstacle_background_frame)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = diagram.plant().GetMyContextFromRoot(context)

    # ['world-hand_x', 'world-hand_y', 'world-hand_qz', 
    # 'theta0_q', 'theta1_q', 'theta2_q', 'theta3_q', 'theta4_q', 
    # 'world-hand_vx', 'world-hand_vy', 'world-hand_wz', 
    # 'theta0_w', 'theta1_w', 'theta2_w', 'theta3_w', 'theta4_w']

    # NOTE: world-hand_x is Z, world-hand_y is X
    plant.SetPositions(plant_context, np.array([0.1, 0.02, 0., 0., 0., 0., 0., 0.]))
    plant.SetVelocities(plant_context, np.array([0., 0., 0., 0., 0., 0., 0., 0.]))

    print(plant.GetStateNames())
    print(plant.get_actuation_input_port().size())
    inspector = scene_graph.model_inspector()

    kMinTimeStep = 0.01
    kMaxTimeStep = 0.1
    hybrid = HybridMultibodyCollocation(diagram, context, kMinTimeStep,
                                    kMaxTimeStep)
    prog = hybrid.prog()

    pair_contacts = hybrid.GetContactPairCandidates()
    print(pair_contacts)
    print(len(pair_contacts))

    contacts = [None, None, None] # middle bar, top bar, ladder back
    for contact in pair_contacts:
        id1, id2 = contact
        print(inspector.GetName(id1), inspector.GetName(id2))
        contact_name = inspector.GetName(id2)
        if (contact_name == "Obstacle::obstacle_middle_arm"):
            contacts[0] = contact
        if (contact_name == "Obstacle::obstacle_top_arm"):
            contacts[1] = contact
        if (contact_name == "Obstacle::obstacle_base_collision"):
            contacts[2] = contact

    # initially falling onto first bar
    kNumTimeSteps_aerial_init = 5
    aerial_init = hybrid.AddMode("aerial_init", kNumTimeSteps_aerial_init, set())
    aerial_init.AddEqualTimeIntervalsConstraints()

    initial_state = [0.1, 0.02, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    prog.AddBoundingBoxConstraint(initial_state, initial_state, aerial_init.initial_state())

    # joint limits
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[4] >= -2.8)
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[4] <= 0.)
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[5] >= -0.3)
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[5] <= 3.4)
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[6] >= 0.)
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[6] <= 2.8)
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[7] >= -2.8)
    aerial_init.AddConstraintToAllKnotPoints(aerial_init.state()[7] <= 0.)

    u_aerial_init = aerial_init.input()

    # ADD COSTS AND CONSTRAINTS FOR AERIAL INIT


    # catching middle bar, sliding down to hit back
    kNumTimeSteps_middle_bar_contact = 2
    middle_bar_contact = hybrid.AddModeWithInelasticImpact(
        "middle_bar_contact", kNumTimeSteps_middle_bar_contact, contacts[0])
    middle_bar_contact.AddEqualTimeIntervalsConstraints()

    # joint limits
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[4] >= -2.8)
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[4] <= 0.)
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[5] >= -0.3)
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[5] <= 3.4)
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[6] >= 0.)
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[6] <= 2.8)
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[7] >= -2.8)
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[7] <= 0.)

    u_middle_bar_contact = middle_bar_contact.input()

    # ADD COSTS AND CONSTRAINTS FOR MIDDLE BAR CONTACT
    middle_bar_contact.AddConstraintToAllKnotPoints(middle_bar_contact.state()[0] >= 0.05)


    # hitting back of ladder initially
    kNumTimeSteps_middle_back_bar_contact = 2
    middle_back_bar_contact = hybrid.AddModeWithInelasticImpact(
        "middle_back_bar_contact", kNumTimeSteps_middle_back_bar_contact, contacts[2])
    middle_back_bar_contact.AddEqualTimeIntervalsConstraints()

    # joint limits
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[4] >= -2.8)
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[4] <= 0.)
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[5] >= -0.3)
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[5] <= 3.4)
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[6] >= 0.)
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[6] <= 2.8)
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[7] >= -2.8)
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[7] <= 0.)

    u_middle_back_bar_contact = middle_back_bar_contact.input()

    # ADD COSTS AND CONSTRAINTS FOR MIDDLE-BACK BAR CONTACT
    middle_back_bar_contact.AddConstraintToAllKnotPoints(middle_back_bar_contact.state()[0] >= 0.05)


    # hitting back of ladder, turn into joint
    kNumTimeSteps_middle_bar_grab = 10
    middle_bar_grab = hybrid.AddMode(
        "middle_bar_grab", kNumTimeSteps_middle_bar_grab, {contacts[0], contacts[2]})
    middle_bar_grab.AddEqualTimeIntervalsConstraints()

    # joint limits
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[4] >= -2.8)
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[4] <= 0.)
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[5] >= -0.3)
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[5] <= 3.4)
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[6] >= 0.)
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[6] <= 2.8)
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[7] >= -2.8)
    middle_bar_grab.AddConstraintToAllKnotPoints(middle_bar_grab.state()[7] <= 0.)

    u_middle_bar_grab = middle_bar_grab.input()
    x_middle_bar_grab = middle_bar_grab.state()
    x_middle_bar_grab_final = middle_bar_grab.final_state()

    # ADD COSTS AND CONSTRAINTS FOR MIDDLE BAR GRAB
    middle_bar_grab.AddConstraintToAllKnotPoints(x_middle_bar_grab[0] >= 0)

    Q_middle_bar_grab = 100
    Q_middle_bar_grab_final = 10
    middle_bar_grab.AddRunningCost(Q_middle_bar_grab * 
                                    (x_middle_bar_grab[12]**2  
                                    + x_middle_bar_grab[13]**2
                                    + x_middle_bar_grab[14]**2 
                                    + x_middle_bar_grab[15]**2))

    middle_bar_grab.AddFinalCost(Q_middle_bar_grab_final * 
                                    (x_middle_bar_grab_final[12]    # theta1 to swing faster cw  (negative)
                                    - x_middle_bar_grab_final[13]   # theta2 to swing faster ccw (positive)
                                    - x_middle_bar_grab_final[14]   # theta3 to swing faster ccw
                                    + x_middle_bar_grab_final[15])) # theta4 to swing faster cw

    # middle_bar_grab.AddFinalCost(Q_middle_bar_grab_final * (
    #         (x_middle_bar_grab_final[3])**2 + (x_middle_bar_grab_final[4] - (-2.8))**2 
    #         + (x_middle_bar_grab_final[5] - (2.8))**2 + (x_middle_bar_grab_final[6] - (2.0))**2 
    #         + (x_middle_bar_grab_final[7] - (-2.0))**2
    # )) # body should be curled up at the end
    final_state_lower = [-1000., -1000., -1000., 0., -2.8, 2.0, 1.6, -2.0,
                         -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000.]
    final_state_upper = [1000., 1000., 1000., 0., -2.0, 2.8, 2.0, -1.6,
                         1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.]
    prog.AddBoundingBoxConstraint(final_state_lower, final_state_upper, middle_bar_grab.final_state()) # body should be curled up at the end!!!


    # let go, get rid of sticking contacts
    # kNumTimeSteps_middle_bar_release = 2
    # middle_bar_release = hybrid.AddModeWithInelasticImpact(
    #     "middle_bar_release", kNumTimeSteps_middle_bar_release, contacts[0])
    # middle_bar_release.AddEqualTimeIntervalsConstraints()

    # u_middle_bar_release = middle_bar_release.input()

    # # ADD COSTS AND CONSTRAINTS FOR MIDDLE BAR RELEASE
    # middle_bar_release.AddConstraintToAllKnotPoints(middle_bar_release.state()[0] >= 0.05)


    # let go, launching robot up - above first rung
    kNumTimeSteps_aerial_launch_1 = 3
    aerial_launch_1 = hybrid.AddMode("aerial_launch_1", kNumTimeSteps_aerial_launch_1, set())
    aerial_launch_1.AddEqualTimeIntervalsConstraints()

    # joint limits
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[4] >= -2.8)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[4] <= 0.)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[5] >= -0.3)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[5] <= 3.4)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[6] >= 0.)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[6] <= 2.8)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[7] >= -2.8)
    aerial_launch_1.AddConstraintToAllKnotPoints(aerial_launch_1.state()[7] <= 0.)

    u_aerial_launch_1 = aerial_launch_1.input()
    x_aerial_launch_1 = aerial_launch_1.state()

    # ADD COSTS AND CONSTRAINTS FOR AERIAL LAUNCH 1
    aerial_launch_1.AddConstraintToAllKnotPoints(x_aerial_launch_1[0] >= 0.09)
    aerial_launch_1.AddConstraintToAllKnotPoints(x_aerial_launch_1[0] <= 0.39)
    aerial_launch_1.AddConstraintToAllKnotPoints(x_aerial_launch_1[1] <= -0.02)

    # Q_aerial_launch_1 = 10
    # aerial_launch_1.AddRunningCost(Q_aerial_launch_1 * 
    #                             ((x_aerial_launch_1[0]-0.6)**2 + (x_aerial_launch_1[1]-0.02)**2))
    
    # let go, launching robot up - outside of ladder
    kNumTimeSteps_aerial_launch_2 = 4
    aerial_launch_2 = hybrid.AddMode("aerial_launch_2", kNumTimeSteps_aerial_launch_2, set())
    aerial_launch_2.AddEqualTimeIntervalsConstraints()

    # joint limits
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[4] >= -2.8)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[4] <= 0.)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[5] >= -0.3)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[5] <= 3.4)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[6] >= 0.)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[6] <= 2.8)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[7] >= -2.8)
    aerial_launch_2.AddConstraintToAllKnotPoints(aerial_launch_2.state()[7] <= 0.)

    u_aerial_launch_2 = aerial_launch_2.input()
    x_aerial_launch_2 = aerial_launch_2.state()

    # ADD COSTS AND CONSTRAINTS FOR AERIAL LAUNCH 2
    aerial_launch_2.AddConstraintToAllKnotPoints(x_aerial_launch_2[1] <= -0.19)
    aerial_launch_2.AddConstraintToAllKnotPoints(x_aerial_launch_2[1] >= -1.0)
    aerial_launch_2.AddConstraintToAllKnotPoints(x_aerial_launch_2[0] >= 0.09)
    aerial_launch_2.AddConstraintToAllKnotPoints(x_aerial_launch_2[0] <= 1)

    # Q_aerial_launch_2 = 10
    # aerial_launch_2.AddRunningCost(Q_aerial_launch_2 * 
    #                             ((x_aerial_launch_2[0]-0.6)**2 + (x_aerial_launch_2[1]-0.02)**2))

    
    # let go, launching robot up - above second rung
    kNumTimeSteps_aerial_launch_3 = 3
    aerial_launch_3 = hybrid.AddMode("aerial_launch_3", kNumTimeSteps_aerial_launch_3, set())
    aerial_launch_3.AddEqualTimeIntervalsConstraints()

    # joint limits
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[4] >= -2.8)
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[4] <= 0.)
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[5] >= -0.3)
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[5] <= 3.4)
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[6] >= 0.)
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[6] <= 2.8)
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[7] >= -2.8)
    aerial_launch_3.AddConstraintToAllKnotPoints(aerial_launch_3.state()[7] <= 0.)

    # ends in position right above top bar
    final_state = [0.6, 0.02, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    prog.AddBoundingBoxConstraint(final_state, final_state, aerial_launch_3.final_state())

    u_aerial_launch_3 = aerial_launch_3.input()
    x_aerial_launch_3 = aerial_launch_3.state()

    # ADD COSTS AND CONSTRAINTS FOR AERIAL LAUNCH 3
    aerial_launch_3.AddConstraintToAllKnotPoints(x_aerial_launch_3[0] >= 0.59)
    aerial_launch_3.AddConstraintToAllKnotPoints(x_aerial_launch_3[0] <= 1)

    # Q_aerial_launch_3 = 10
    # aerial_launch_3.AddRunningCost(Q_aerial_launch_3 * 
    #                             ((x_aerial_launch_3[0]-0.6)**2 + (x_aerial_launch_3[1]-0.02)**2))

    
    # catching top bar, sliding down to hit back
    kNumTimeSteps_top_bar_contact = 2
    top_bar_contact = hybrid.AddModeWithInelasticImpact(
        "top_bar_contact", kNumTimeSteps_top_bar_contact, contacts[1])
    top_bar_contact.AddEqualTimeIntervalsConstraints()

    # joint limits
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[4] >= -2.8)
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[4] <= 0.)
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[5] >= -0.3)
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[5] <= 3.4)
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[6] >= 0.)
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[6] <= 2.8)
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[7] >= -2.8)
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[7] <= 0.)
    
    u_top_bar_contact = top_bar_contact.input()

    # ADD COSTS AND CONSTRAINTS FOR TOP BAR CONTACT
    top_bar_contact.AddConstraintToAllKnotPoints(top_bar_contact.state()[0] >= 0.55)


    # hitting back of ladder initially
    kNumTimeSteps_top_back_bar_contact = 2
    top_back_bar_contact = hybrid.AddModeWithInelasticImpact(
        "top_back_bar_contact", kNumTimeSteps_top_back_bar_contact, contacts[2])
    top_back_bar_contact.AddEqualTimeIntervalsConstraints()

    # joint limits
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[4] >= -2.8)
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[4] <= 0.)
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[5] >= -0.3)
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[5] <= 3.4)
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[6] >= 0.)
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[6] <= 2.8)
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[7] >= -2.8)
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[7] <= 0.)

    u_top_back_bar_contact = top_back_bar_contact.input()

    # ADD COSTS AND CONSTRAINTS FOR TOP-BACK BAR CONTACT
    top_back_bar_contact.AddConstraintToAllKnotPoints(top_back_bar_contact.state()[0] >= 0.55)


    # hitting back of ladder, turn into joint
    kNumTimeSteps_top_bar_grab = 2
    top_bar_grab = hybrid.AddMode(
        "top_bar_grab", kNumTimeSteps_top_bar_grab, {contacts[1], contacts[2]})
    top_bar_grab.AddEqualTimeIntervalsConstraints()

    # joint limits
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[4] >= -2.8)
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[4] <= 0.)
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[5] >= -0.3)
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[5] <= 3.4)
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[6] >= 0.)
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[6] <= 2.8)
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[7] >= -2.8)
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[7] <= 0.)
    
    u_top_bar_grab = top_bar_grab.input()
    
    # ADD COSTS AND CONSTRAINTS FOR TOP BAR GRAB
    top_bar_grab.AddConstraintToAllKnotPoints(top_bar_grab.state()[0] >= 0.55)


    result = Solve(prog)
                        
    traj = hybrid.ReconstructStateTrajectory(result)
    force = hybrid.ReconstructContactForceTrajectory(
        result, hybrid.GetContactPairCandidates().pop())

    print('all phases [time, state, contact force]')
    print(np.vstack((traj.get_segment_times(), 
                    traj.vector_values(traj.get_segment_times()),
                    force.vector_values(traj.get_segment_times()))).T)
    
    x_trajectory = traj
    u_trajectory = hybrid.ReconstructInputTrajectory(result)

    if not result.is_success():
        print(result.GetInfeasibleConstraintNames(prog))

    return traj, u_trajectory

x_trajectory, u_trajectory = HybridClimber()

['world-hand_x', 'world-hand_y', 'world-hand_qz', 'theta0_q', 'theta1_q', 'theta2_q', 'theta3_q', 'theta4_q', 'world-hand_vx', 'world-hand_vy', 'world-hand_wz', 'theta0_w', 'theta1_w', 'theta2_w', 'theta3_w', 'theta4_w']
4
{(<GeometryId value=1202>, <GeometryId value=1280>), (<GeometryId value=1202>, <GeometryId value=1283>), (<GeometryId value=1202>, <GeometryId value=1267>)}
3
Climber::Sphere Obstacle::obstacle_top_arm
Climber::Sphere Obstacle::obstacle_middle_arm
Climber::Sphere Obstacle::obstacle_base_collision
all phases [time, state, contact force]
[[  0.      0.1     0.02    0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.   ]
 [  0.011   0.099   0.019   0.      0.009  -0.02    0.01    0.018  -0.043
   -0.133  -0.342  -0.      2.928  -6.601   4.163   2.243  -7.294   0.
    0.      0.   ]
 [  0.021   0.097   0.014  -0.      0.043  -0.089   0.047   0.047  -0.13
   -0.221  -0.433   0.      1.228   0.512

## Simulate

In [10]:
def create_scene(x_traj_source, sim_time_step=0.0001):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant = MultibodyPlant(time_step=0.0)
    scene_graph = SceneGraph()
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)

    # Loading models.
    # Load the climber 
    parser.AddModelsFromString(climber, "urdf")

    # Load the obstacle
    parser.AddModelsFromString(obstacle_background, "urdf")
    parser.AddModelsFromString(obstacle, "urdf")

    #Weld the obstacle to the world so that it's fixed during the simulation.
    obstacle_frame = plant.GetFrameByName("obstacle_base")
    plant.WeldFrames(plant.world_frame(), obstacle_frame)
    obstacle_background_frame = plant.GetFrameByName("obstacle_base_background")
    plant.WeldFrames(plant.world_frame(), obstacle_background_frame)

    # Finalize the plant after loading the scene.
    plant.Finalize()
    # We use the default context to calculate the transformation of the table
    # in world frame but this is NOT the context the Diagram consumes.
    plant_context = plant.CreateDefaultContext()

    plant.SetPositions(plant_context, np.array([0.1, 0.02, 0., 0., 0., 0., 0., 0.]))
    plant.SetVelocities(plant_context, np.array([0., 0., 0., 0., 0., 0., 0., 0.]))

    builder.AddSystem(x_traj_source)
    builder.AddSystem(scene_graph)
    pos_to_pose = builder.AddSystem(
        MultibodyPositionToGeometryPose(plant, input_multibody_state=True)
    )

    builder.Connect(x_traj_source.get_output_port(0), pos_to_pose.get_input_port())
    builder.Connect(
        pos_to_pose.get_output_port(),
        scene_graph.get_source_pose_port(plant.get_source_id()),
    )

    # Add visualizer to visualize the geometries.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))

    diagram = builder.Build()
    return diagram, visualizer

In [17]:
x_traj_source = TrajectorySource(x_trajectory)

def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(x_traj_source, sim_time_step):
    diagram, visualizer = create_scene(x_traj_source, sim_time_step)
    simulator = initialize_simulation(diagram)
    visualizer.StartRecording()
    simulator.AdvanceTo(2)
    visualizer.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(x_traj_source, sim_time_step=0.0001)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=067a6c9a-b930-4c28-8084-e86f86716a80' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>